# Imports

In [1]:
import pandas as pd
import boto3
from os import environ as env
from dotenv import load_dotenv

# Importação dos Dados

In [2]:
df_raw = pd.read_csv('../data/anuario-2024-dados_abertos-tabela2-19.csv', sep=';')

In [3]:
df_raw.head()

,BENEFICIÁRIO,ANO,DISTRIBUIÇÃO DE ROYALTIES SOBRE A PRODUÇÃO DE PETRÓLEO E DE GÁS NATURAL
0,Alagoas (Estado),2014,36993
1,Alagoas (Estado),2015,"27547,54381"
2,Alagoas (Estado),2016,"22606,50137"
3,Alagoas (Estado),2017,"22204,34408"
4,Alagoas (Estado),2018,"24711,96505"


# Exploração de Dados

In [4]:
df = df_raw.copy()

In [5]:
df.shape

(370, 3)

In [6]:
df.dtypes

BENEFICIÁRIO                                                               object
ANO                                                                         int64
DISTRIBUIÇÃO DE ROYALTIES SOBRE A PRODUÇÃO DE PETRÓLEO E DE GÁS NATURAL    object
dtype: object

In [7]:
df.isna().sum()

BENEFICIÁRIO                                                               0
ANO                                                                        0
DISTRIBUIÇÃO DE ROYALTIES SOBRE A PRODUÇÃO DE PETRÓLEO E DE GÁS NATURAL    0
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df.columns

Index(['BENEFICIÁRIO', 'ANO',
       'DISTRIBUIÇÃO DE ROYALTIES SOBRE A PRODUÇÃO DE PETRÓLEO E DE GÁS NATURAL'],
      dtype='object')

# Formulação de Perguntas 

1. Qual a diferença total (somando todos os anos) no recebimento dos royalties entre cada estado e os seus municípios?
2. Que região do Brasil recebeu mais royalties e que região recebeu menos?
3. Qual foi a evolução de royalties para beneficiários que não os estados ou os municípios desses estados?

# Exportação dos Dados com boto3

In [10]:
load_dotenv()

access_key = env.get('AWS_ACCESS_KEY_ID')
secret_key = env.get('AWS_SECRET_ACCESS_KEY')
bucket_name = env.get('BUCKET_NAME')

if not all([access_key, secret_key, bucket_name]):
    missing = [name for name, value in [
        ('AWS_ACCESS_KEY_ID', access_key),
        ('AWS_SECRET_ACCESS_KEY', secret_key),
        ('BUCKET_NAME', bucket_name)
    ] if not value]
    raise ValueError(f"Variáveis de ambiente faltando: {', '.join(missing)}")

s3 = boto3.client(
    's3',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name='us-east-1',
)

try:
    s3.head_bucket(Bucket=bucket_name)
    print(f'Bucket encontrado')

    df.to_csv(
        f's3://{bucket_name}/anuario-2024-dados_abertos-tabela2-19.csv',
        storage_options={
            'key': access_key,
            'secret': secret_key
        },
        index=False
    )
    print('Upload realizado')

except Exception as e:
    print('Erro:', e)

Bucket encontrado
Upload realizado
